<a href="https://colab.research.google.com/github/NafizIR/Gcollab/blob/I-comse/Wednesday/Day3_GCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#First, make sure the notebook is aware of the workshop data sets
!git clone https://github.com/icomse/9th_workshop_ml_for_molecules.git
import os
os.chdir('9th_workshop_ml_for_molecules/data')

Cloning into '9th_workshop_ml_for_molecules'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 246 (delta 66), reused 39 (delta 35), pack-reused 142 (from 1)
Receiving objects: 100% (246/246), 29.38 MiB | 34.50 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [3]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.1 MB/s eta 0:00:00


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [1]:
!mamba install -c conda-forge rdkit


Looking for: ['rdkit']

[+] 0.0s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   2%
conda-forge/noarch     3%[+] 0.3s
conda-forge/linux-64   8%
conda-forge/noarch    18%[+] 0.4s
conda-forge/linux-64  20%
conda-forge/noarch    45%[+] 0.5s
conda-forge/linux-64  31%
conda-forge/noarch    67%[+] 0.6s
conda-forge/linux-64  40%
conda-forge/noarch    87%conda-forge/noarch                                
[+] 0.7s
conda-forge/linux-64  54%[+] 0.8s
conda-forge/linux-64  62%[+] 0.9s
conda-forge/linux-64  72%[+] 1.0s
conda-forge/linux-64  79%[+] 1.1s
conda-forge/linux-64  87%[+] 1.2s
conda-forge/linux-64  94%conda-forge/linux-64                              

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  Updating specs:

   - rdkit
   - ca-certificates
   - certifi
   - openssl


  Package                  

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from rdkit import Chem

import torch

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

## Some Useful Functions:

In [5]:
# I usually keep these in a utils.py file and import them into my other files:
def smiles2geodata(smi,y, feat_dict):
    """
    Inputs:
    smi = smiles string
    y = target
    feat_dict = dictionary where keys are atoms and values are feature array (constructed using get_atom_features function)

    Outputs:
    geo_dp = pytorch geometric datapoint object
    """
    mol = Chem.MolFromSmiles(smi)
    atomic_nums = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    atom_features = torch.tensor([feat_dict[x] for x in atomic_nums]).float()
    edges = get_edge_indices(mol)
    geo_dp = Data(x=atom_features, edge_index=edges,y=y)

    return geo_dp


def get_edge_indices(mol):
    """
    Inputs:
    mol = rdkit molecule object

    Outputs:
    torch tensor containing edge list (for inputting into smiles2geodata)
    """

    edges =[]
    for bond in mol.GetBonds():
        edges.append((bond.GetBeginAtomIdx(),bond.GetEndAtomIdx()))

    edges = [[x[0] for x in edges],[x[1] for x in edges]]

    return torch.tensor(edges,dtype=torch.long)


def get_atom_features(smi_list):
    """
    One-hot encodes atom types and constructs a dictionary to convert atom types into feature vectors

    Inputs:
    smi_list = list of all smiles in the dataset

    Outputs:
    feat_dict = dictionary where keys are atoms and values are feature array
    """

    atom_types = []
    for smi in smi_list:
        mol = Chem.MolFromSmiles(smi)
        atom_types.extend([atom.GetAtomicNum() for atom in mol.GetAtoms()])

    atom_set = list(set(atom_types))

    enc = OneHotEncoder()
    enc.fit(np.array(atom_set).reshape(-1,1))
    feat_dict = {x:enc.transform([[x]]).toarray()[0] for x in atom_set}

    return feat_dict

### Building a deep learning model in Pytorch requires building multiple objects and routines:

1. <u>Dataset</u>- object that contains your training/validation data

2. <u>Model</u>- object that contains your model architecture and weights

3. <u>Train Routine</u>- function that passes training data through model and updates weights

4. <u>Validation Routine</u>- function that passes validation through model and reports performance

## 1. Dataset

In [7]:
data_path = 'aqsol.csv'
df = pd.read_csv(data_path)
df.head()

,SMILES,Solubility
0,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127
1,O=C1Nc2cccc3cccc1c23,-3.254767
2,Clc1ccc(C=O)cc1,-2.177078
3,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409
4,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065


In [ ]:
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

In [ ]:
class GeoDataset(InMemoryDataset):
    def __init__(self, root='./',raw_name=data_path,processed_name='lipo_processed.pt',transform=None, pre_transform=None):

        self.filename = os.path.join(root,raw_name)
        # self.processed_filename = os.path.join(root,processed_name)

        # read a csv from that path:
        self.df = pd.read_csv(self.filename)

        # assign dataset attribute "input_vectors" to be the 2048 bit vector representing each molecule:
        self.x = self.df[self.df.columns[0]].values

        # assign dataset attribute "output_targets" to be the scalar representing binding strength (last column):
        self.y = self.df[self.df.columns[-1]].values


        super(GeoDataset, self).__init__(root, transform, pre_transform)

        self.data, self.slices = torch.load(self.processed_paths[0],weights_only=False)


    def processed_file_names(self):
        return ['data.pt']

    def process(self):

        feat_dict = get_atom_features(self.x)

        data_list = [smiles2geodata(x,y,feat_dict) for x,y in zip(self.x,self.y)]

        data, slices = self.collate(data_list)

        torch.save((data, slices), self.processed_paths[0])


## 2. Model

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import aggr

In [ ]:
class GCN_Geo(torch.nn.Module):

    def __init__(self, hidden_dim_gcn=256, hidden_dim_fcn=256):
        super(GCN_Geo, self).__init__()

        self.conv1 = GCNConv(12, hidden_dim_gcn)
        self.conv2 = GCNConv(hidden_dim_gcn, hidden_dim_gcn)

        self.readout = aggr.SumAggregation()

        self.linear1 = nn.Linear(hidden_dim_gcn, hidden_dim_fcn)
        self.linear2 = nn.Linear(hidden_dim_fcn, 1)

    def forward(self, data):

        # Message passing layers:
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Aggregate to get molecule level features:
        x= self.readout(x,data.batch)

        # FCNN to predict molecular property:
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)

        return x.view(-1,)

## Pulling it all together

In [ ]:
from torch_geometric.loader import DataLoader

In [ ]:
device = torch.device('cpu')
model = GCN_Geo().to(device)

dataset = GeoDataset()
dataloader = DataLoader(dataset,batch_size=32,shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = torch.nn.MSELoss()

In [ ]:
model.train()
for epoch in range(30):
    print('Epoch: {}'.format(epoch))
    for batch in dataloader:

        optimizer.zero_grad()

        out = model(batch)

        loss = loss_fn(out.double(), batch.y.double())

        loss.backward()

        optimizer.step()

In [ ]:
def predict(model, dataloader):

    # Set our model to evaluation mode:
    model.eval()

    X_all = []
    y_all = []
    pred_all = []

    # Remove gradients:
    with torch.no_grad():

        # Looping over the dataloader allows us to pull out or input/output data:
        for batch in dataloader:

            # Make a prediction:
            pred = model(batch)

            X_all.append(batch.x)
            y_all.append(batch.y)
            pred_all.append(pred)

    X_all = torch.concat(X_all)
    y_all = torch.concat(y_all)
    pred_all = torch.concat(pred_all)

    return X_all, y_all, pred_all

In [ ]:
x,y,pred = predict(model,dataloader)

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(y,pred,alpha=0.4)
plt.plot([-1,4],[-1,4],color='k')
plt.show()

In [ ]:
r2_score(y,pred)

***HACKING:*** We didn't include training/validation splits! Can you go back and including training and validation splits, using the FCNN code from the previous section as a reference?